In [105]:
import torch
import cv2
import numpy as np
import pyrealsense2 as rs
import pandas as pd
import import_ipynb
from Angle import *

In [106]:

class ObjectDection:
    def __init__(self,device,isRealsense=False):
        self.device = device
        self.model = self.load_model()
        self.isRealsense = isRealsense


    def load_model(self):
        model = torch.hub.load('ultralytics/yolov5', 'custom',path = "./best.pt")
        return model
    
    def camera_setup(self):
        pipeline = rs.pipeline()
        pipeline_wrapper = rs.pipeline_wrapper(pipeline)
        config = rs.config()
        pipeline_profile = config.resolve(pipeline_wrapper)
        device = pipeline_profile.get_device()
        device_product_line = str(device.get_info(rs.camera_info.product_line))

        config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
        config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

        # Start streaming
        pipeline.start(config)
        return pipeline
     

    def capture(self):
        if self.isRealsense:
            pipeline = self.camera_setup()

        cap= cv2.VideoCapture(self.device)
        ##cap.set(cv2.CAP_PROP_FRAME_HEIGHT,480)
        ##cap.set(cv2.CAP_PROP_FRAME_WIDTH,640)
        
        while True:
            if self.isRealsense:
                ret,color_image,depth_image = self.get_frame(pipeline)
            else:
                ret,color_image = cap.read()
                
            if ret:
                filtered_image  = self.get_blue_frame(color_image)

                result_object = self.detect_object(filtered_image)
                cords = self.write_result(result_object,color_image)
                self.write_angle_offset(color_image,cords)
                """
                if self.isRealsense== False:
                    result_object_right = self.detect_object(color_image_2)
                    cords_right = self.write_result(result_object_right,color_image_2)
                    distances = self.get_depth(cords,cords_right)
                    self.write_result_depth(color_image,cords,distances)
                """
                cv2.imshow("frame",color_image)
                key = cv2.waitKey(1)
                if key == 27 or key == ord("q"):
                    break
        print("Finish")
        cap.release()
        cv2.destroyAllWindows()
        


    def detect_object(self,color_frame):
        #color_frame = [color_frame]
        results = self.model(color_frame)
        #labels,cord = results.xyxyn[0][:,-1], results.xyxyn[0][:,:-1]

        return results

    def detect_depth(self,depth_image,point):
            return depth_image[point[1], point[0]]


    
    def __call__(self):
        self.capture()

    def get_frame(self,pipeline):
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        if not depth_frame or not color_frame:
            return False, None, None
        return True, depth_image, color_image


    def get_depth(self,cords_left,cords_right):
        x_center = 640/2 # The camera I used is 480p, 640 pixels * 480 pixels, the center is 640/2
        y_center = 480/2
        
        cam_distance = 13 # distance between two cameras cm
        fov = 90 # fov 90 degrees
        length = min(len(cords_left),len(cords_right))
        
        f_pixel = x_center / np.tan(fov * 0.5 * np.pi/180) # find the pixel length cm/
        distance = []
        for i in range(length):
            disparity = cords_left[i][0]-cords_right[i][1] # diparity in pixels
            zDepth = f_pixel * cam_distance / disparity
            distance.append(abs(zDepth))

        return distance
    
    def get_blue_frame(self,frame):
        
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        lower_blue = np.array([110,50,50])
        upper_blue = np.array([130,255,255])
        mask =  cv2.inRange(hsv,lower_blue,upper_blue)
        contours,hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            area = cv2.contourArea(contour)
            if area >450:
                x_min,y_min,width,height = cv2.boundingRect(contour)
                ##print(x_min,y_min,width,height)
                cv2.rectangle(mask,(x_min-30,y_min-30),(x_min+width+30,y_min+height+30),(2552,255,255),-1)
        res = cv2.bitwise_and(frame,frame,mask = mask)

        return res
    
    def write_angle_offset(self,frame,cords):
        offset_calculator = Angle(70,42,640,360) ## Here the x_fov of intel D435i is 70 and y_fov is 42, the pixel of the video is 640* 360 
        for cord in cords:
            x_offset = offset_calculator.get_horizontal_offset(cord[0])
            y_offset = offset_calculator.get_vertical_offset(cord[1])
            cv2.putText(frame,str(round(x_offset))+" "+str(round(y_offset)),cord,cv2.FONT_HERSHEY_COMPLEX_SMALL,0.5,(0, 255, 0),1,cv2.LINE_AA)

    def write_result(self,results,frame):
        confidence_threshold = 0.25
        
        result_to_pandas = results.pandas().xyxy[0]
        results = result_to_pandas.to_numpy()
       
        num_of_objects = len(result_to_pandas)
        cords = []
        for i in range(num_of_objects):
            
            if results[i,4] > confidence_threshold:
                x_min,y_min,x_max,y_max = int(results[i,0]),int(results[i,1]),int(results[i,2]),int(results[i,3])
                cv2.rectangle(frame,(x_min,y_min),(x_max,y_max),(0,255,0),2)

                cv2.putText(frame,str(results[i,6]),(x_min,y_min),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.5,(0, 255, 0),1,cv2.LINE_AA)
                cords.append((round((x_min+x_max)/2),round((y_min+y_max)/2)))
        return cords
    
    def write_result_depth(self,frame,cords,distances):
        for i in range(len(distances)):
            cv2.putText(frame,format(distances[i],'.2f'),cords[i],cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,0.5,(0,255,0),1,cv2.LINE_AA)
    
    def write_depth(self,frame,depth_image,cords):
        for cord in cords:
            distance = self.detect_depth(depth_image,cord)
            cv2.putText(frame,distance,cord,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2,cv2.LINE_AA)


In [107]:
print(torch.__version__)
print(format(1.23423,'.2f'))


1.12.1
1.23


In [108]:
object_detect  = ObjectDection("vid.mp4",False) # Since I don't have intel d435i, so I put a false here to use my alternative method to detect distance
# Put Ture as a parameter if the camera is intel d435i
object_detect()

Using cache found in C:\Users\admin/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-7-1 Python-3.10.9 torch-1.12.1 CPU

Fusing layers... 
Model summary: 213 layers, 1760518 parameters, 0 gradients
Adding AutoShape... 


Finish


In [109]:
print("1"+" ")


1 


In [110]:
"""
cap=cv2.VideoCapture("vid.mp4")

while True:
  
    ret,frame = cap.read()
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    lower_blue = np.array([110,50,50])
    upper_blue = np.array([130,255,255])
    mask =  cv2.inRange(hsv,lower_blue,upper_blue)
    contours,hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    print(frame.shape)
    roi = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area >450:
            x_min,y_min,width,height = cv2.boundingRect(contour)
            ##print(x_min,y_min,width,height)
            cv2.rectangle(mask,(x_min-30,y_min-30),(x_min+width+30,y_min+height+30),(2552,255,255),-1)
            ##mask[x_min+30:x_min+width-30,y_min+30:y_min+height-30] = 255
           ## roi.append([(x_min-30,y_min-30),(x_min+width+30,y_min+height+30)])
            cv2.rectangle(frame,(x_min-30,y_min-30),(x_min+width+30,y_min+height+30),(0,255,0),3)

    res = cv2.bitwise_and(frame,frame,mask = mask)

    cv2.imshow("test",frame)
    cv2.imshow("hsv",hsv)
    cv2.imshow("mask",mask)
    cv2.imshow("Filted",res)
    key = cv2.waitKey(2)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

"""

'\ncap=cv2.VideoCapture("vid.mp4")\n\nwhile True:\n  \n    ret,frame = cap.read()\n    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)\n    lower_blue = np.array([110,50,50])\n    upper_blue = np.array([130,255,255])\n    mask =  cv2.inRange(hsv,lower_blue,upper_blue)\n    contours,hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)\n    print(frame.shape)\n    roi = []\n    for contour in contours:\n        area = cv2.contourArea(contour)\n        if area >450:\n            x_min,y_min,width,height = cv2.boundingRect(contour)\n            ##print(x_min,y_min,width,height)\n            cv2.rectangle(mask,(x_min-30,y_min-30),(x_min+width+30,y_min+height+30),(2552,255,255),-1)\n            ##mask[x_min+30:x_min+width-30,y_min+30:y_min+height-30] = 255\n           ## roi.append([(x_min-30,y_min-30),(x_min+width+30,y_min+height+30)])\n            cv2.rectangle(frame,(x_min-30,y_min-30),(x_min+width+30,y_min+height+30),(0,255,0),3)\n\n    res = cv2.bitwise_and(frame,f